In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
import os

In [11]:
cv=LeaveOneOut()

In [12]:
#X=pd.read_csv("Kmer4.csv",header=None)
y=pd.read_csv("lable.txt",header=None)
y=np.array(y)
#print(X.shape)

In [13]:
def my_classifier1(X,y):
    print(datetime.datetime.now())
    TN_a=[]
    FN_a=[]
    FP_a=[]
    TP_a=[]
    accuracy_a=[]
    pre_a=[]
    rec_a=[]
    f1_a=[]
    auroc_a=[]
    param1=["best","random"]
    score=0
    best_param=param1[0]
    for i in param1:
        Predict_Y=[]    #存放预测值
        Prob_Y=[]       #存放预测概率
        Really_Y=y
        for train_index,test_index in cv.split(X):
            estimator=DecisionTreeClassifier(random_state=10,splitter=i)
            estimator.fit(X[train_index],y[train_index])
            Predict_Y.append(estimator.predict(X[test_index])[0])
            Prob_Y.append(estimator.predict_proba(X[test_index]))
        Prob_Y=np.array(Prob_Y)  #将Prob_Y改为numpy数组形式
        Prob_Y=Prob_Y.reshape(173,2)  #原本是(173,1,2)形状，改为（173,2）形状
        Prob_Y=pd.DataFrame(Prob_Y)
        Prob_Y.columns=["0","1"]
        confusion_matrix(Really_Y,Predict_Y)
        TN=confusion_matrix(Really_Y,Predict_Y)[0,0]
        FN=confusion_matrix(Really_Y,Predict_Y)[1,0]
        FP=confusion_matrix(Really_Y,Predict_Y)[0,1]
        TP=confusion_matrix(Really_Y,Predict_Y)[1,1]
        ACC=accuracy_score(Really_Y,Predict_Y)
        Precision=precision_score(Really_Y,Predict_Y)
        Recall=recall_score(Really_Y,Predict_Y)
        F1_score=f1_score(Really_Y,Predict_Y)
        AUC=roc_auc_score(Really_Y,np.array(Prob_Y["1"]))
        if ACC>score:
            score=ACC
            best_param=i
        TN_a.append(TN)
        FN_a.append(FN)
        FP_a.append(FP)
        TP_a.append(TP)
        accuracy_a.append(ACC)
        pre_a.append(Precision)
        rec_a.append(Recall)
        f1_a.append(F1_score)
        auroc_a.append(AUC)  
    results=np.array([param1,TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a])
    result_final = pd.DataFrame(results)
    return best_param,results

In [14]:
def my_classifier2(p1,X,y):
    print(datetime.datetime.now())
    param2=np.array(range(3,10,1))
   # print(datetime.datetime.now())
    TN_a=[]
    FN_a=[]
    FP_a=[]
    TP_a=[]
    accuracy_a=[]
    pre_a=[]
    rec_a=[]
    f1_a=[]
    auroc_a=[]
    score=0
    best_param=param2[0]
    #param1=["best","random"]
    for i in param2:
        Predict_Y=[]    #存放预测值
        Prob_Y=[]       #存放预测概率
        Really_Y=y
        for train_index,test_index in cv.split(X):
            estimator=DecisionTreeClassifier(random_state=10,splitter=p1,max_depth=i)
            estimator.fit(X[train_index],y[train_index])
            Predict_Y.append(estimator.predict(X[test_index])[0])
            Prob_Y.append(estimator.predict_proba(X[test_index]))
        Prob_Y=np.array(Prob_Y)  #将Prob_Y改为numpy数组形式
        Prob_Y=Prob_Y.reshape(173,2)  #原本是(173,1,2)形状，改为（173,2）形状
        Prob_Y=pd.DataFrame(Prob_Y)
        Prob_Y.columns=["0","1"]
        confusion_matrix(Really_Y,Predict_Y)
        TN=confusion_matrix(Really_Y,Predict_Y)[0,0]
        FN=confusion_matrix(Really_Y,Predict_Y)[1,0]
        FP=confusion_matrix(Really_Y,Predict_Y)[0,1]
        TP=confusion_matrix(Really_Y,Predict_Y)[1,1]
        ACC=accuracy_score(Really_Y,Predict_Y)
        if ACC>score:
            score=ACC
            best_param=i
        Precision=precision_score(Really_Y,Predict_Y)
        Recall=recall_score(Really_Y,Predict_Y)
        F1_score=f1_score(Really_Y,Predict_Y)
        AUC=roc_auc_score(Really_Y,np.array(Prob_Y["1"]))
        TN_a.append(TN)
        FN_a.append(FN)
        FP_a.append(FP)
        TP_a.append(TP)
        accuracy_a.append(ACC)
        pre_a.append(Precision)
        rec_a.append(Recall)
        f1_a.append(F1_score)
        auroc_a.append(AUC)  
    results=np.array([param2,TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a])
    result_final = pd.DataFrame(results)
  #  return grid_search.best_params_["splitter"],results
    return best_param,results

In [15]:
def my_classifier(p1,p2,X, y):
    print(datetime.datetime.now())
   # param2=["best","random"]   ##设置splitter范围
   # param3=np.array(range(3,10,1))   ##设置max_depth范围
    i=X.shape[1]
    param3=np.array(range(1,i,1))  ## 设置max_features范围
    param2=np.array(range(3,10,1))
   # print(datetime.datetime.now())
    TN_a=[]
    FN_a=[]
    FP_a=[]
    TP_a=[]
    accuracy_a=[]
    pre_a=[]
    rec_a=[]
    f1_a=[]
    auroc_a=[]
    #param1=["best","random"]
    for j in param3:
        Predict_Y=[]    #存放预测值
        Prob_Y=[]       #存放预测概率
        Really_Y=y
        for train_index,test_index in cv.split(X):
            estimator=DecisionTreeClassifier(random_state=10,splitter=p1,max_depth=p2,max_features=j)
            estimator.fit(X[train_index],y[train_index])
            Predict_Y.append(estimator.predict(X[test_index])[0])
            Prob_Y.append(estimator.predict_proba(X[test_index]))
        Prob_Y=np.array(Prob_Y)  #将Prob_Y改为numpy数组形式
        Prob_Y=Prob_Y.reshape(173,2)  #原本是(173,1,2)形状，改为（173,2）形状
        Prob_Y=pd.DataFrame(Prob_Y)
        Prob_Y.columns=["0","1"]
        confusion_matrix(Really_Y,Predict_Y)
        TN=confusion_matrix(Really_Y,Predict_Y)[0,0]
        FN=confusion_matrix(Really_Y,Predict_Y)[1,0]
        FP=confusion_matrix(Really_Y,Predict_Y)[0,1]
        TP=confusion_matrix(Really_Y,Predict_Y)[1,1]
        ACC=accuracy_score(Really_Y,Predict_Y)
        Precision=precision_score(Really_Y,Predict_Y)
        Recall=recall_score(Really_Y,Predict_Y)
        F1_score=f1_score(Really_Y,Predict_Y)
        AUC=roc_auc_score(Really_Y,np.array(Prob_Y["1"]))
        TN_a.append(TN)
        FN_a.append(FN)
        FP_a.append(FP)
        TP_a.append(TP)
        accuracy_a.append(ACC)
        pre_a.append(Precision)
        rec_a.append(Recall)
        f1_a.append(F1_score)
        auroc_a.append(AUC)  
    results=np.array([param3,TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a])
    return results

In [17]:
file_path = 'E:\datatest\Mismatch\m1' # 文件夹目录
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=np.loadtxt(path_whole)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
    print(r1.shape)
    print(r2.shape)
    print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
10.txt
**************************************************
2022-02-24 13:05:43.089495
2022-02-24 13:05:43.371766
2022-02-24 13:05:44.361092
(10, 2)
(10, 7)
(10, 3)
(10, 1)
##################################################
20.txt
**************************************************
2022-02-24 13:05:44.748056
2022-02-24 13:05:45.097145
2022-02-24 13:05:46.431552
(10, 2)
(10, 7)
(10, 15)
(10, 1)
##################################################
21.txt
**************************************************
2022-02-24 13:05:49.178224
2022-02-24 13:05:49.577136
2022-02-24 13:05:51.142946
(10, 2)
(10, 7)
(10, 15)
(10, 1)
##################################################
30.txt
**************************************************
2022-02-24 13:05:53.575438
2022-02-24 13:05:54.178845
2022-02-24 13:05:56.360986
(10, 2)
(10, 7)
(10, 63)
(10, 1)
##################################################
31.txt
**************************************************
